# AIVLE스쿨 2차 미니프로젝트: 신규 아파트 주차 수요 예측

<img src = "https://github.com/Jangrae/img/blob/master/parking.png?raw=true" width=800, align="left"/>

# 단계 3: 모델링

## [미션]

- 모델링을 수행합니다.
    - 전처리를 추가로 진행합니다.
    - 4개 이상의 알고리즘을 사용해 모델링을 수행합니다.
    - 각 모델에 대해 성능 튜닝을 수행합니다.
    - 성능을 비교해 최선의 모델을 선정합니다.
- 데이터 파이프라인 함수를 만듭니다.
- 새로운 데이터를 읽어와 예측을 수행합니다.

## 1. 환경설정

### (1) 로컬 수행(Anaconda)

- project 폴더에 필요한 파일들을 넣고, 본 파일을 열었다면, 별도 경로 지정이 필요하지 않습니다.

In [ ]:
# 기본 경로
path = ''

### (2) 구글 콜랩 수행

- 구글 콜랩을 사용중이면 구글 드라이브를 연결합니다.

In [48]:
# 구글 드라이브 연결, 패스 지정
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/MyDrive/project/'

### (3) 한글 폰트 표시용 라이브러리 설치

In [15]:
# 한글 표시를 위한 라이브러리 설치
# !pip install koreanize_matplotlib -q

### (4) 라이브러리 불러오기

In [167]:
# 기본 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns

# 모델링용 라이브러리 불러오기
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import lightgbm as lgb

from sklearn.metrics import *

# 기타 라이브러리 불러기기
import joblib
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format='retina'

### (5) 데이터 불러오기

- joblib.dump() 함수를 사용해 base_data2.pkl 파일을 읽어옵니다.
- 읽어온 데이터를 data 데이터프레임으로 선언하고 기본 정보를 확인합니다.

In [63]:
# 파일 읽어오기
data = joblib.load('base_data2.pkl')

# 확인
data.head()

,총세대수,준공연도,건물형태,난방방식,승강기설치여부,실차량수,총면적,10-30,30-40,40-50,50-60,60-70,70-80,80-200,임대보증금,임대료
0,78,2013,계단식,개별,1,109,6023.7683,0,0,0,78,0,0,0,5.696200e+07,642930.000000
1,35,2013,복도식,개별,1,35,1569.1668,35,0,0,0,0,0,0,6.306200e+07,470100.000000
2,88,2013,계단식,개별,1,88,7180.1396,0,0,0,88,0,0,0,7.219000e+07,586540.000000
3,477,2014,복도식,지역,1,943,47058.9273,0,0,0,150,0,216,111,1.015167e+08,950305.000000
4,15,2013,복도식,개별,1,21,543.0268,15,0,0,0,0,0,0,5.522750e+07,340148.333333


In [65]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345 entries, 0 to 344
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   총세대수     345 non-null    int64  
 1   준공연도     345 non-null    int32  
 2   건물형태     345 non-null    object 
 3   난방방식     345 non-null    object 
 4   승강기설치여부  345 non-null    int64  
 5   실차량수     345 non-null    int64  
 6   총면적      345 non-null    float64
 7   10-30    345 non-null    int64  
 8   30-40    345 non-null    int64  
 9   40-50    345 non-null    int64  
 10  50-60    345 non-null    int64  
 11  60-70    345 non-null    int64  
 12  70-80    345 non-null    int64  
 13  80-200   345 non-null    int64  
 14  임대보증금    345 non-null    float64
 15  임대료      345 non-null    float64
dtypes: float64(3), int32(1), int64(10), object(2)
memory usage: 41.9+ KB


### (6) 함수 생성

- 실젯값과 모델이 예측한 값을 시각화해 비교할 함수를 만듭니다.

In [73]:
# 실젯값, 예측값 비교 함수 만들기
def model_plot(y_test, y_pred):
     plt.figure(figsize=(10, 3))
     plt.plot(y_test.values, label='Actual', linewidth=0.7, marker='o', markersize=2)
     plt.plot(y_pred, label='Predicted', linewidth=0.7, marker='o', markersize=2)
     plt.legend()
     plt.show()

## 2. 모델링

- 모델링을 위한 결측치 처리, 데이터 분할, 스케일링, 가변수화 등을 수행합니다.
- 4개 이상의 알고리즘을 사용하여 모델을 만듭니다.
- 모델 이름은 서로 다르게 합니다. (예: model1, model2...)
- Linear Regression 이외의 알고리즘을 사용한 모델은 GridSearchCV() 함수로 성능 최적화를 수행합니다.
- 적절한 평가지표로 모델의 성능을 평가합니다.
- 실젯값과 예측값을 시각화해 비교합니다.
- 성능 비교를 통해 최선의 모델을 선정합니다.

### (1) 데이터 전처리

- 필요한 전처리를 수행합니다.

#### 1) 가변수화

- '건물형태', '난방방식' 변수에 대해 가변수화를 수행합니다.

In [87]:
# '건물형태', '난방방식' 변수를 가변수화 (One-Hot Encoding)
data_dummies = pd.get_dummies(data, columns=['건물형태', '난방방식'], drop_first=True)
# Boolean 타입을 0과 1로 변환
data_dummies = data_dummies.astype(int)
# 결과 확인
data_dummies.head()

,총세대수,준공연도,승강기설치여부,실차량수,총면적,10-30,30-40,40-50,50-60,60-70,70-80,80-200,임대보증금,임대료,건물형태_복도식,건물형태_혼합식,난방방식_중앙,난방방식_지역
0,78,2013,1,109,6023,0,0,0,78,0,0,0,56962000,642930,0,0,0,0
1,35,2013,1,35,1569,35,0,0,0,0,0,0,63062000,470100,1,0,0,0
2,88,2013,1,88,7180,0,0,0,88,0,0,0,72190000,586540,0,0,0,0
3,477,2014,1,943,47058,0,0,0,150,0,216,111,101516666,950305,1,0,0,1
4,15,2013,1,21,543,15,0,0,0,0,0,0,55227500,340148,1,0,0,0


#### 2) x, y 분리

- Target를 지정하고, x와 y로 분리합니다.

In [120]:
target='실차량수'
x=data_dummies.drop(columns=target)
y=data_dummies.loc[:,target]

#### 3) 학습용, 평가용 분리

- 학습용, 평가용 데이터를 적절한 비율로 분리합니다.

In [122]:
# 학습용, 평가용 데이터 분리
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1)

#### 4) 스케일링

- 필요한 경우 스케일링을 진행합니다.
- 예를 들어 KNN 알고리즘을 사용할 경우입니다.

### (2) 모델 1: 선형회귀

In [124]:
# 5. 선형 회귀 모델 생성 및 학습
model = LinearRegression()
model.fit(x_train, y_train)

# 6. 예측
y_pred = model.predict(x_test)

# 7. 모델 평가
mse = mean_squared_error(y_test, y_pred)  # MSE (평균 제곱 오차)
rmse = np.sqrt(mse)  # RMSE (루트 평균 제곱 오차)
r2 = r2_score(y_test, y_pred)  # R² 스코어

# 8. 결과 출력
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared: {r2:.2f}")

Mean Squared Error: 45674.65
Root Mean Squared Error: 213.72
R-squared: 0.66


Mean Squared Error (MSE): 45,674.65

의미: 모델이 예측한 값과 실제 값 간의 평균 오차의 제곱합입니다. 값이 클수록 모델의 예측 오차가 큽니다.
Root Mean Squared Error (RMSE): 213.72

의미: 예측 오차의 표준 편차로, 단위가 실제 데이터와 동일하여 해석이 쉽습니다. 예를 들어, 예측값이 평균적으로 실제값과 약 214대 정도 차이가 난다고 볼 수 있습니다.
R-squared (R²): 0.66

의미: 0.66은 약 66%의 설명력을 가진다는 뜻입니다. 즉, 모델이 입력 데이터에 있는 정보의 66%를 설명할 수 있고 나머지 34%는 설명하지 못하는 것입니다.
좋은 R² 값: 일반적으로 0.7 이상이 좋은 설명력을 가지며, 0.9 이상이면 매우 높은 설명력을 의미합니다.

### (3) 모델 2: 랜덤포레스트


In [130]:
# 5. 랜덤 포레스트 회귀 모델 생성 및 학습
rf_model = RandomForestRegressor(n_estimators=100, random_state=1)
rf_model.fit(x_train, y_train)

# 6. 예측
y_pred = rf_model.predict(x_test)

# 7. 모델 평가
mse = mean_squared_error(y_test, y_pred)  # MSE (평균 제곱 오차)
rmse = np.sqrt(mse)  # RMSE (루트 평균 제곱 오차)
r2 = r2_score(y_test, y_pred)  # R² 스코어

# 8. 결과 출력
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared: {r2:.2f}")

Mean Squared Error: 41365.84
Root Mean Squared Error: 203.39
R-squared: 0.69


Mean Squared Error (MSE): 41,365.84

의미: 모델의 예측값과 실제값 간의 평균 제곱 오차로, 예측 오차의 크기를 나타냅니다. 이전의 선형 회귀 모델보다 낮은 값으로, 모델의 예측 정확도가 향상되었음을 보여줍니다.
Root Mean Squared Error (RMSE): 203.39

의미: 예측 오차의 표준 편차로, 예측값이 실제값과 평균적으로 약 203대 차이가 난다는 것을 의미합니다. RMSE 또한 이전 모델보다 개선된 수치입니다.
R-squared (R²): 0.69

의미: 모델이 입력 데이터의 변동성 중 약 69%를 설명할 수 있음을 나타냅니다. 이는 꽤 좋은 설명력이며, 모델이 상당한 비율의 변동성을 잘 잡아내고 있다는 것을 의미합니다.

### (4) 모델 3: GridSearchCV를 사용한 랜덤 포레스트 모델

In [139]:
# 5. 랜덤 포레스트 모델 생성
rf_model = RandomForestRegressor(random_state=1)

# 6. 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 7. GridSearchCV 설정
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid,
                           scoring='neg_mean_squared_error', cv=5, verbose=2, n_jobs=-1)

# 8. 하이퍼파라미터 튜닝 및 모델 학습
grid_search.fit(x_train, y_train)

# 9. 최적의 파라미터와 최적의 모델
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# 10. 예측
y_pred = best_model.predict(x_test)

# 11. 모델 평가
mse = mean_squared_error(y_test, y_pred)  # MSE (평균 제곱 오차)
rmse = np.sqrt(mse)  # RMSE (루트 평균 제곱 오차)
r2 = r2_score(y_test, y_pred)  # R² 스코어

# 12. 결과 출력
print(f"Best Parameters: {best_params}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared: {r2:.2f}")


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
Mean Squared Error: 40753.86
Root Mean Squared Error: 201.88
R-squared: 0.69


Mean Squared Error (MSE): 40,753.86

의미: 모델의 예측값과 실제값 간의 평균 제곱 오차입니다. 이전 결과보다 약간 낮아져, 예측 성능이 향상되었음을 나타냅니다.
Root Mean Squared Error (RMSE): 201.88

의미: 예측값이 실제값과 평균적으로 약 202대 차이가 난다는 것을 의미합니다.
R-squared (R²): 0.69

의미: 모델이 입력 데이터의 변동성 중 약 69%를 설명할 수 있다는 것을 나타냅니다. 이전 결과와 유사한 수준으로, 안정적인 성능을 보여줍니다.

### (5) 모델 4: XGBoost

In [149]:
# 5. XGBoost 모델 생성
xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=1)

# 6. 모델 학습
xgb_model.fit(x_train, y_train)

# 7. 예측
y_pred = xgb_model.predict(x_test)

# 8. 모델 평가
mse = mean_squared_error(y_test, y_pred)  # MSE (평균 제곱 오차)
rmse = np.sqrt(mse)  # RMSE (루트 평균 제곱 오차)
r2 = r2_score(y_test, y_pred)  # R² 스코어

# 9. 결과 출력
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared: {r2:.2f}")

Mean Squared Error: 40742.80
Root Mean Squared Error: 201.85
R-squared: 0.69


Mean Squared Error (MSE): 40,742.80

의미: 모델의 예측값과 실제값 간의 평균 제곱 오차입니다. 이전 랜덤 포레스트 모델과 비슷한 수준입니다.
Root Mean Squared Error (RMSE): 201.85

의미: 예측값과 실제값 간의 평균적인 차이를 나타내며, 약 202대의 차이가 난다는 것을 의미합니다. RMSE 또한 이전 모델과 유사한 수준입니다.
R-squared (R²): 0.69

의미: 모델이 데이터의 변동성을 약 69% 설명할 수 있다는 것을 나타냅니다. 이는 안정적인 성능을 보여줍니다.
모델 비교
XGBoost와 랜덤 포레스트의 성능이 비슷하게 나타났습니다. 두 모델 모두 R² 값이 0.69로 상당히 유사하며, MSE와 RMSE도 비슷한 수준입니다. 이는 데이터의 특성과 예측하고자 하는 목표 변수의 관계에 따라 이 두 모델이 유사한 성능을 발휘하고 있음을 시사합니다.

### (6) 모델 5: LightGBM

In [164]:
# 5. LightGBM 모델 생성
lgb_model = lgb.LGBMRegressor(n_estimators=100, random_state=1)

# 6. 모델 학습
lgb_model.fit(x_train, y_train)

# 7. 예측
y_pred = lgb_model.predict(x_test)

# 8. 모델 평가
mse = mean_squared_error(y_test, y_pred)  # MSE (평균 제곱 오차)
rmse = np.sqrt(mse)  # RMSE (루트 평균 제곱 오차)
r2 = r2_score(y_test, y_pred)  # R² 스코어

# 9. 결과 출력
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared: {r2:.2f}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000439 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 512
[LightGBM] [Info] Number of data points in the train set: 241, number of used features: 14
[LightGBM] [Info] Start training from score 565.518672
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

### (7) 성능 비교

- 각 모델의 성능을 비교합니다.

## 3. 파이프라인 구축

- 새로운 데이터를 불러오고, 이 데이터를 처리할 파이프라인 함수를 만듭니다.

### (1) New Data 불러오기

- test.xlsx 파일을 읽어와 new_data 데이터프레임으로 선언합니다.
- 해당 데이터는 '실차량수' 변수가 없는 것 외에는, 최초 데이터와 동일한 구조입니다.
- 이 데이터를 대상으로 전처리와 예측을 수행합니다.

In [ ]:
# 파일 읽어오기
new_data = pd.read_excel(path+'test.xlsx')

# 확인
new_data.head()

### (2) 데이터 파이프라인 구축

- 데이터 파이프라인 함수를 만듭니다.
- 학습 데이터에 대해 진행했던 모든 전처리 과정을 평가 데이터에도 일괄 진행해야 합니다.
    - 입력: new_data
    - 출력: 전처리가 완료된 예측 직전 데이터프레임
- 새로운 데이터에는 '실차량수' 변수가 없음을 유의합니다.
- 참고: 다음 내용들이 처리되어야 합니다.
    - 결측치 처리
    - 변수 추가
    - 불필요한 변수 제거
    - 단지 데이터, 상세 데이터 분리
    - 단지코드별 총면적 합 집계
    - 전용면적 구간별 집계 (피벗 형태)
    - 임대보증금, 임대료 평균 집계
    - 집계 결과 병합
    - 난방방식: 개별, 지역, 중앙 세 가지로 묶기
    - 승강기설치여부: 0, 1 값으로 변경
    - 단지모드, 지역 변수 제거
    - 가변수화

In [ ]:
# 파이프라인 만들기
def data_pipeline(data):
    apt01 = data.copy()

    

### (3) 예측하기

- new_data를 파이프라인을 사용해 전처리한 후 가장 성능이 좋았던 모델로 예측한 결과를 확인합니다.

In [ ]:
# 데이터 전처리
data = data_pipeline(new_data)

# 확인
data.head()

In [ ]:
# 예측하기
predicted = model4.predict(data)

# 확인
print(predicted)

- 아파트 기본 정보에 예측한 차량수를 붙여 마무리합니다.

In [ ]:
# 데이터 셋 두개로 나누기
vars = ['단지코드', '단지명', '총세대수', '지역', ]
result = new_data[vars].copy()
result = result.drop_duplicates()
result.reset_index(drop=True, inplace=True)

# 예측 결과 추가
result['예상차량수'] = predicted.round(1).astype(int)

# 확인
result